<a href="https://colab.research.google.com/github/Akshay-ramagiri/NNDL/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORTING REQUIRED PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##IMPORT DATA

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Assignment3.csv')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,0:00,5551.82208
1,NaN,1:00,4983.17184
2,NaN,2:00,4888.39680
3,NaN,3:00,5072.95872
4,NaN,4:00,5196.25980


In [ ]:
X = []
Y = []
for i in range(1, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-1]) 
    Y.append(data['Load (kW)'].iloc[i])

In [ ]:
data = pd.DataFrame({'Previous Day Load' : X, 'Present Day Load' : Y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


##NORMALIZATION

In [ ]:
Xmax = np.max(data['Previous Day Load'])
Xmin = np.min(data['Previous Day Load'])
Ymax = np.max(data['Present Day Load'])
Ymin = np.min(data['Present Day Load'])
data['Previous Day Load'] = (data['Previous Day Load'] - Xmin) / (Xmax - Xmin)
data['Present Day Load'] = (data['Present Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Previous Day Load,Present Day Load
0,0.397877,0.293800
1,0.293800,0.276454
2,0.276454,0.310234
3,0.310234,0.332801
4,0.332801,0.414254


##DATA SPLIT

In [ ]:
x = data['Previous Day Load']
y = data['Present Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

##GRADIENT CALCULATION AND MODEL TRAINING

In [ ]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1
for _ in range(epochs):   
    for i in range(Xtrain.shape[0]):
        tempM = m + v*vm
        tempC = c + v*vc       
        gM = -1 * (Ytrain.iloc[i] - tempM * Xtrain.iloc[i] - tempC) * Xtrain.iloc[i]
        gC = -1 * (Ytrain.iloc[i] - tempM * Xtrain.iloc[i] - tempC)
        vm = (v * vm) - (eta * gM)
        vc = (v * vc) - (eta * gC)
        m += vm
        c += vc
print(f'm = {m}\nc = {c}')

m = 0.7799247301408083
c = 0.11205495172826244


In [ ]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

##DENORMALIZATION

In [ ]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

##ERROR METRIC CALCULATION
TRAINING DATA

In [ ]:
train_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
train_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
train_RMSE = train_MSE ** 0.5
print('Training Error :')
print(f'MAE : {train_MAE}')
print(f'MSE : {train_MSE}')
print(f'RMSE : {train_RMSE}')

Training Error :
MAE : 470.7854451595653
MSE : 430838.4062024668
RMSE : 656.3828198562686


TESTING DATA

In [ ]:
test_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
test_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
test_RMSE = test_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {test_MAE}')
print(f'MSE : {test_MSE}')
print(f'RMSE : {test_RMSE}')

Testing Error :
MAE : 516.1635560852437
MSE : 542768.9860455303
RMSE : 736.7285701298208


##Analyze training data predictions

In [ ]:
pd.DataFrame({'Actual' : trainY, 'Predicted':train_pred})

,Actual,Predicted
0,7272.11376,6798.534635
1,6082.74936,6550.279165
2,6436.59696,6624.682862
3,5349.80160,6263.606101
4,8121.65976,7451.390597
...,...,...
1959,5671.84968,5749.102110
1960,5442.86196,5957.967388
1961,4870.93824,5495.011056
1962,8138.65068,7769.429925


##Analyze testing data predictions

In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted':test_pred})

,Actual,Predicted
0,6052.82040,5875.782913
1,5190.80400,5700.958542
2,5097.27600,5466.319435
3,6263.57016,6892.147129
4,4543.90200,4979.777618
...,...,...
214,6172.84800,6517.697155
215,5864.20560,5325.900695
216,4667.82660,4228.689326
217,7943.64480,7748.032784


##PREDICTION

In [ ]:
x = float(input('Enter the load at previous hour : '))
x = (x - Xmin) / (Xmax - Xmin)
pred = m * x + c
pred = (pred * (Ymax - Ymin)) + Ymin
print('The Predicted load at present hour = ', pred)

Enter the load at previous hour : 5000
The Predicted load at present hour =  5255.260447295394
